<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/FCN_FCN_PLUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
# Load the training and test data
train_path = '/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy'
test_path = '/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy'
train_data = np.load(train_path)
test_data = np.load(test_path)

# Extract the sensor data and the target variables (S and D)
X_train = train_data[:, :1000]
y_train = train_data[:, -2:]
X_test = test_data[:, :1000]
y_test = test_data[:, -2:]

In [ ]:
X_train_mean = np.mean(X_train, axis = 1)
X_train_mean = np.reshape(X_train_mean, (-1,1))
X_train_std = np.std(X_train, axis = 1)
X_train_std = np.reshape(X_train_std, (-1,1))
X_train = (X_train - X_train_mean)/X_train_std


X_test_mean = np.mean(X_test, axis = 1)
X_test_mean = np.reshape(X_test_mean, (-1,1))
X_test_std = np.std(X_test, axis = 1)
X_test_std = np.reshape(X_test_std, (-1,1))
X_test = (X_test - X_test_mean)/X_test_std


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.model_selection import train_test_split

# # Load the complete training and testing data
# train_data = np.load('simu_20000_0.1_90_140_train.npy')
# test_data = np.load('simu_10000_0.1_141_178_test.npy')

# Separate features and labels
X_train_full = train_data[:, :1000]
y_train_full = train_data[:, -2]  # Only Systolic Blood Pressure
X_test = test_data[:, :1000]
y_test = test_data[:, -2]  # Only Systolic Blood Pressure

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.3, random_state=42, shuffle=True)

# Scale the features
X_train_mean = np.mean(X_train, axis=1)
X_train_mean = np.reshape(X_train_mean, (-1, 1))
X_train_std = np.std(X_train, axis=1)
X_train_std = np.reshape(X_train_std, (-1, 1))
X_train = (X_train - X_train_mean) / X_train_std

X_val_mean = np.mean(X_val, axis=1)
X_val_mean = np.reshape(X_val_mean, (-1, 1))
X_val_std = np.std(X_val, axis=1)
X_val_std = np.reshape(X_val_std, (-1, 1))
X_val = (X_val - X_val_mean) / X_val_std

X_test_mean = np.mean(X_test, axis=1)
X_test_mean = np.reshape(X_test_mean, (-1, 1))
X_test_std = np.std(X_test, axis=1)
X_test_std = np.reshape(X_test_std, (-1, 1))
X_test = (X_test - X_test_mean) / X_test_std

# Reshape data for FCN_PLUS
X_train = X_train[:, :, np.newaxis]
X_val = X_val[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

# Build the FCN_PLUS model
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1], 1)),
    layers.Conv1D(filters=128, kernel_size=8, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv1D(filters=256, kernel_size=5, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv1D(filters=128, kernel_size=3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(1)  # Single output node for Systolic Blood Pressure
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError()])

# Train the model for 70 epochs and capture loss after each epoch
for epoch in range(50):
    history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val), shuffle=False, verbose=1)

    train_loss = history.history['loss'][0]
    train_mae = history.history['mean_absolute_error'][0]

    val_loss = history.history['val_loss'][0]
    val_mae = history.history['val_mean_absolute_error'][0]

    # Evaluate the model on the test set
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Train MAE: {train_mae}, Val Loss: {val_loss}, Val MAE: {val_mae}, Test Loss: {test_loss}, Test MAE: {test_mae}")
    prediction = model.predict(X_test)
    mae_systolic = MeanAbsoluteError()
    mae_systolic.update_state(y_test, prediction)
    print("Mean absolute error for systolic is: ",mae_systolic.result().numpy())
